In [1]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# convolution - > apply kernel filter to part of image
# stride -> size to move before reapply convolution
# zero-padding -> pad zero at the side of image
# Max-pooling -> pick largest in a patch (for regular)
# avg-pooling (subsampling) -> avg in a patch
# fully connected neural net -> linear layer

In [3]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Processing...
Done!


In [4]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # (in_channel,out_channel,kernel_size)
        # kernel size is the height x width of filter
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [6]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [7]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
for epoch in range(1, 10):
    train(epoch)
    test()

/Users/setup/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306707
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.290851
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.222578
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.203814
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.137507
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.047542
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.982659
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.747416
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.598474
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.253501
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.024615
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.986720
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.685456
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.743571
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.626444
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.690592
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.424161
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.534021
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.760594
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.706132
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.079067
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.378746
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.149025
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.189864
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.117462
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.226081
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.177476
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.064346
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.106971
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.134841
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.077615
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.103046
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.103926
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.069132
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.183232
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.108501
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.220709
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.159461
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.140625
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.040010


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.068733
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.097486
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.021282
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.189450
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.113310
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.070960
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.042953
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.050105
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.240710
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.017434
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.262133
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.126410
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.161110
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.108788
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.173310
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.040357
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.075601
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.099570
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.225261
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.127559


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.040065
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.044365
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.046391
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.165295
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.136786
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.089275
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.066006
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.082401
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.066515
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.191393
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.106608
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.089656
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.116406
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.052681
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.035392
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.094551
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.087180
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.012298
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.082239
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.098163


Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.132548

Test set: Average loss: 0.0619, Accuracy: 9802/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.016036
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.139957
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.070806
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.169690
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.019665
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.015934
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.022397
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.066232
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.068819
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.072356
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.028189
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.044039
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.137712
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.011463
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.102270
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.052343
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.099376
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.057173
Train Epoch: 8 [11

Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.104489
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.193525
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.098711
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.057913
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.007517
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.049570
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.052152
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.051536
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.042160
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.069696
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.078455
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.047542
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.016281
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.007702
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.107087
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.016385
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.108474
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.025981
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.035822
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.049333
